In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from tqdm.notebook import tqdm
import seaborn as sns


from sklearn.metrics import confusion_matrix
from sklearn import model_selection

import torch
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import transforms
from torchvision.utils import make_grid
import torch.nn.functional as F

In [ ]:
from google.colab import files
 
 
uploaded = files.upload()

Saving Aerial.csv to Aerial.csv


In [ ]:
FULL_DATA_DF = ('Aerial.csv')


IDX_CLASS_LABELS = {
    0: 'ceoTREES_CANOPYCOVER',
    1: 'ceoBUSH_SCRUB', 
    2: 'ceoWaterTreatedPool',
    3: 'ceoGRASS',
    4: 'ceoCultivatedVeg',
    5: 'ceoWaterLakePondedContainer',
    6: 'ceoShadow',
    7: 'ceoBareGround',
    8: 'ceoBuilding',
    9: 'ceoImperviousSurface'
}
CLASSES = ['ceoTREES_CANOPYCOVER', 'ceoBUSH_SCRUB', 'ceoWaterTreatedPool', 'ceoGRASS', 'ceoCultivatedVeg', 'ceoWaterLakePondedContainer','ceoShadow','ceoBareGround','ceoBuilding', 'ceoImperviousSurface']
CLASS_IDX_LABELS = dict()
for key, val in IDX_CLASS_LABELS.items():
  CLASS_IDX_LABELS[val] = key

NUM_CLASSES = len(IDX_CLASS_LABELS.items())
torch.manual_seed(10)
VALID_SIZE = 0.1

In [ ]:
## Give idx of each class name
def encode_label(label):
    idx = CLASS_IDX_LABELS[label] 
    return idx

## Take in idx and return the class name
def decode_target(target, text_labels=True):
    result = []
    if text_labels:
        return IDX_CLASS_LABELS[target]
    else:
        return target



In [ ]:
print(encode_label('ceoWaterTreatedPool'), decode_target(2))
print(decode_target(2, text_labels=True))

2 ceoWaterTreatedPool
ceoWaterTreatedPool


In [ ]:
DATA_DF = pd.read_csv(FULL_DATA_DF)
DATA_DF = DATA_DF.sample(frac = 1, random_state=48) 
TRAIN_DF = DATA_DF[:-int(len(DATA_DF)*VALID_SIZE)]
TEST_DF = DATA_DF[-int(len(DATA_DF)*VALID_SIZE) :]

In [ ]:
TRAIN_DF.reset_index(inplace = True) 
TRAIN_DF.head()

,index,ceoPLOTID,LONGITUDE,LATITUDE,SIZE_M,PLOTSHAPE,ceoFLAGGED,SAMPLE_POINTS,seesAOIplot,AOI_NUM,ceoPLOT_NUM,ceoTREES_CANOPYCOVER,ceoBUSH_SCRUB,ceoGRASS,ceoCultivatedVeg,ceoWaterTreatedPool,ceoWaterLakePondedContainer,ceoWaterRiverStream,ceoWaterIrrigation,ceoShadow,ceoUnknown,ceoBareGround,ceoBuilding,ceoImperviousSurface
0,834,137375063,-95.695165,29.712992,100,square,False,121,5407,54,7,94.214876,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,5.785124,0.000000,0.000000,0.000000,0.000000
1,97,137373318,-84.275866,33.761913,100,square,False,121,526,5,26,55.371901,0.000000,19.834711,0.000000,0.0,0.0,0.0,0.0,1.652893,0.826446,0.000000,0.000000,22.314050
2,574,137374263,-84.340181,34.032794,100,square,False,121,3135,31,35,0.000000,0.000000,11.570248,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,24.793388,63.636364
3,242,137373535,-95.416617,29.784612,100,square,False,121,1127,11,27,0.000000,57.024793,18.181818,0.000000,0.0,0.0,0.0,0.0,4.132231,0.000000,19.008264,0.000000,1.652893
4,1565,137376658,-97.824134,30.448419,100,square,False,121,9818,98,18,57.024793,2.479339,14.049587,7.438017,0.0,0.0,0.0,0.0,0.000000,0.000000,0.826446,12.396694,5.785124


In [ ]:
VALID_DF.reset_index(inplace = True) 
VALID_DF.head()

,level_0,index,ceoPLOTID,LONGITUDE,LATITUDE,SIZE_M,PLOTSHAPE,ceoFLAGGED,SAMPLE_POINTS,seesAOIplot,AOI_NUM,ceoPLOT_NUM,ceoTREES_CANOPYCOVER,ceoBUSH_SCRUB,ceoGRASS,ceoCultivatedVeg,ceoWaterTreatedPool,ceoWaterLakePondedContainer,ceoWaterRiverStream,ceoWaterIrrigation,ceoShadow,ceoUnknown,ceoBareGround,ceoBuilding,ceoImperviousSurface
0,0,1453,137376546,-75.063103,42.456815,100,square,False,121,9514,95,14,15.702479,0.000000,21.487603,0.000000,0.000000,0.0,0.0,0.0,3.305785,0.000000,0.0,42.148760,17.355372
1,1,445,137374062,-96.744901,33.086418,100,square,False,121,2614,26,14,14.049587,0.000000,19.008264,0.000000,0.826446,0.0,0.0,0.0,0.000000,0.000000,0.0,16.528926,49.586777
2,2,195,137373452,-73.735686,40.804060,100,square,False,121,916,9,16,0.000000,0.000000,0.000000,47.933884,0.000000,0.0,0.0,0.0,7.438017,0.000000,0.0,8.264463,36.363636
3,3,1266,137375999,-122.217297,47.759832,100,square,False,121,8007,80,7,40.495868,0.826446,14.049587,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.826446,0.0,19.008264,24.793388
4,4,883,137375292,-74.429303,40.416480,100,square,False,121,6020,60,20,38.842975,0.000000,0.000000,22.314050,0.826446,0.0,0.0,0.0,0.000000,0.000000,0.0,14.049587,23.966942


In [ ]:
TRAIN_DF.size, VALID_DF.size

(38880, 4500)

In [ ]:
class FULL_DATA_DF:
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform
        
    def __len__(self):
        return len(self.train_df)
    

In [ ]:
data_transform = transforms.Compose([
                                transforms.Resize(size=(224, 224)),
#                                 transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])
train_ds = FULL_DATA_DF(TRAIN_DF, data_transform)
valid_ds = FULL_DATA_DF(VALID_DF, data_transform)
print(len(train_ds), len(valid_ds))

1620 180


In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim = 1)
    return torch.tensor(torch.sum(preds==labels).item() / len(preds))

class MulticlassClassifierBase(nn.Module):
    
    def training_step(self, batch):
        label = batch
        loss = criterion(out, label)
        accu = accuracy(out, label)
        return accu ,loss
    def validation_step(self, batch):
        label = batch
        loss = criterion(out, label)
        accu = accuracy(out, label)
        return {"val_loss": loss.detach(), "val_acc": accu}
    
    def validation_epoch_ends(self, outputs):
        batch_loss = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_loss).mean()
        batch_acc = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_acc).mean()
        return {"val_loss":epoch_loss.item(), "val_acc":epoch_acc.item()}
    def epoch_end(self, epoch, result):
        print("Epoch [{}],train_accu: {:.4f}, learning_rate: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch,result['train_accu'], result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
class LULC_Model(MulticlassClassifierBase):
    def __init__(self):
        super().__init__()
        self.network = models.wide_resnet50_2(pretrained=True)
        n_inputs = self.network.fc.in_features
        self.network.fc = nn.Sequential(
                              nn.Linear(n_inputs, 256),
                              nn.ReLU(),
                              nn.Dropout(0.5),
                              nn.Linear(256, NUM_CLASSES),
                              nn.LogSoftmax(dim=1)
                                )
    def forward(self, xb):
        return self.network(xb)
    
    def freeze(self):
        for param in self.network.parameters():
            param.require_grad=False
        for param in self.network.fc.parameters():
            param.require_grad=True
    def unfreeze(self):
        for param in self.network.parameters():
            param.require_grad=True

In [ ]:
model = LULC_Model()

Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth


  0%|          | 0.00/132M [00:00<?, ?B/s]

In [ ]:
@torch.no_grad()
def evaluate(model, valid_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in valid_loader]
    return model.validation_epoch_ends(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def fit(epochs, max_lr,  model, train_loader, valid_loader, weight_decay=0,\
                grad_clip=None,opt_func=torch.optim.SGD, max_epochs_stop=3):
  
    history = []
    valid_loss_min = np.Inf
    valid_acc_max = 0
    model_file_name = 'lulc.pth'
    model_file_name2 = 'lulc_max_acc.pth'
    epochs_no_improve =  0
    optimizer = opt_func(model.parameters(), lr=max_lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.00001)
                         
    for epoch in range(epochs):
        model.train()
        train_loss = []
        train_accu = []
        lrs = []
        for batch in tqdm(train_loader):
            
            accu, loss = model.training_step(batch)
            train_loss.append(loss)
            train_accu.append(accu)
            loss.backward()
            ## Gradient Clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            lrs.append(get_lr(optimizer))
            
            
            
        result = evaluate(model, valid_loader)
        scheduler.step(result['val_loss'])
        ########### Early Stopping ##############                                         
        valid_loss = result['val_loss']
        valid_acc = result['val_acc']
        if valid_acc > valid_acc_max:
            torch.save(model.state_dict(), model_file_name2)
            valid_acc_max = valid_acc
        if valid_loss<valid_loss_min:
            torch.save(model.state_dict(), model_file_name)
            valid_loss_min = valid_loss                                  
            epochs_no_improve = 0          
        else:
            epochs_no_improve += 1
            if epochs_no_improve > max_epochs_stop:
                result["train_loss"] = torch.stack(train_loss).mean().item()
                result["train_accu"] = torch.stack(train_accu).mean().item()
                result["lrs"] = lrs
                model.epoch_end(epoch, result)
                history.append(result)
                print("Early Stopping............................")
                return history                                
                                                 
        result["train_loss"] = torch.stack(train_loss).mean().item()
        result["train_accu"] = torch.stack(train_accu).mean().item()
        result["lrs"] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    print("VAL LOSS MIN {}".format(valid_loss_min))
    print("VAL ACC MAX {}".format(valid_acc_max))
    return history

In [ ]:
## This didnt work too well- have to get images history = fit(epochs, max_lr, model, train_ds, valid_ds)

In [ ]:
def get_device():
    print(torch.cuda.is_available())
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
            
    def __len__(self):
        return len(self.dl)

In [ ]:
print(TRAIN_DF)

      index  ceoPLOTID  ...  ceoBuilding  ceoImperviousSurface
0       834  137375063  ...     0.000000              0.000000
1        97  137373318  ...     0.000000             22.314050
2       574  137374263  ...    24.793388             63.636364
3       242  137373535  ...     0.000000              1.652893
4      1565  137376658  ...    12.396694              5.785124
...     ...        ...  ...          ...                   ...
1615    640  137374437  ...     0.000000              0.000000
1616    938  137375383  ...     0.000000              0.000000
1617    743  137374540  ...    42.148760             13.223140
1618    502  137374191  ...     0.000000              9.090909
1619   1044  137375489  ...     0.000000             17.355372

[1620 rows x 24 columns]


In [ ]:
TRAIN_DF=np.arange(0,len(TRAIN_DF),1)